In [1]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
import tensorflow as tf
import pandas as pd

In [2]:
teste = pd.read_csv("datasets/Classification/Corona_NLP_test.csv")
treino = pd.read_csv("datasets/Classification/Corona_NLP_train.csv",encoding='latin-1')

data = pd.concat([teste,treino])
data = data.sample(frac=0.2, random_state=42)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8991 entries, 29256 to 21571
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       8991 non-null   int64 
 1   ScreenName     8991 non-null   int64 
 2   Location       7135 non-null   object
 3   TweetAt        8991 non-null   object
 4   OriginalTweet  8991 non-null   object
 5   Sentiment      8991 non-null   object
dtypes: int64(2), object(4)
memory usage: 491.7+ KB


In [3]:
data = data.drop(columns=[ "UserName","ScreenName","Location","TweetAt"], axis=1)

In [4]:
def categorize_sentiment(score):
    if score == 'Negative':
        return "Negative"
    elif score == 'Extremely Negative':
        return "Negative"
    elif score == 'Positive':
        return "Positive"
    elif score == 'Extremely Positive':
        return "Positive"
    else:
        return "Neutral"
    
    
data['Sentiment'] = data['Sentiment'].apply(categorize_sentiment)

In [5]:
# Fazer TOKENIZACAO das palavras

import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def get_all_str(tweet):
    sentence = ''
    for words in tweet:
        sentence += words
    sentence = sentence.lower()
    return sentence
def get_str(lst):
    sentence = ''
    for char in lst:
        sentence += char+' '
    sentence = sentence.lower()
    return sentence

def get_word(text): 
    result = nltk.RegexpTokenizer(r'\w+').tokenize(text.lower())
    return result

def get_hashtag(text):
    result = nltk.RegexpTokenizer(r'(?<=#)\w+').tokenize(text.lower())
    return result

def get_mention(text):
    result = nltk.RegexpTokenizer(r'(?<=@)\w+').tokenize(text.lower())
    return result 

def stopword_list(stop):
    lst = stopwords.words('english')
    for stopword in stop:
        lst.append(stopword)
    return lst

def remove_stopword(stopwords, lst):    
    stoplist = stopword_list(stopwords)
    txt = ''
    for idx in range(len(lst)):
        txt += lst[idx]
        txt += '\n'
    cleanwordlist = [word for word in txt.split() if word not in stoplist] 
#     print(stoplist)
    return cleanwordlist

def pos_remove_noun(tagged):
    remove_noun = [word for word,pos in tagged if pos not in ['NN','NNS','NNP','NNPS']]
    txt = ''
    for i in range(len(remove_noun)):
        txt += remove_noun[i]
        txt += '\n'
    return cleanwordlist

def lemmatization(words):
    lemm = WordNetLemmatizer()
    tokens = [lemm.lemmatize(word) for word in words]
#     txt= ' '.join(tokens)
    return tokens

In [7]:
def preprocessing(text):
    words = get_word(text)
    lemma = lemmatization(words)
    removed = remove_stopword(['http','ly'],lemma)
    return removed

data['OriginalTweet'] = data['OriginalTweet'].apply(preprocessing)
data['OriginalTweet'] = data['OriginalTweet'].apply(get_str)

In [8]:
X = data.drop(columns=["Sentiment"])
y = data["Sentiment"]

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer


Tfidf = TfidfVectorizer(stop_words='english', norm='l2' )
X = Tfidf.fit_transform(X["OriginalTweet"]).toarray()


In [10]:
from sklearn.preprocessing import  LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [11]:
# from keras.utils import to_categorical
# y = to_categorical(y, )

In [12]:
y

array([0, 2, 0, ..., 0, 2, 2])

In [13]:
# Load the Boston Housing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
# Scale the data

scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [17]:
# # Build the model
# from keras.models import Sequential
# from keras.layers import Dense, BatchNormalization, Dropout, LayerNormalization
# from keras.initializers import glorot_uniform

# model = Sequential()
# model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_initializer=glorot_uniform() ))
# model.add(Dropout(0.2))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(3, activation='sigmoid'))  # Softmax para classificação de 3 classes


In [18]:
from keras.optimizers import Adam, SGD
from keras.layers import Dense, BatchNormalization, Dropout, LayerNormalization
from keras.initializers import glorot_uniform
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X.shape[1], activation='relu', kernel_initializer=glorot_uniform()))
    model.add(Dropout(0.2, seed=42))
    model.add(LayerNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(np.unique(y)), activation='softmax'))
    model.compile(optimizer=Adam(0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# Perform cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model = create_model()
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=0)
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Loss: {loss}, Accuracy: {accuracy}')

57/57 [==============================] - 1s 9ms/step - loss: 1.5400 - accuracy: 0.6620
Loss: 1.5400452613830566, Accuracy: 0.6620344519615173
57/57 [==============================] - 1s 11ms/step - loss: 1.4039 - accuracy: 0.7047
Loss: 1.4039355516433716, Accuracy: 0.7046718597412109
57/57 [==============================] - 1s 12ms/step - loss: 1.4985 - accuracy: 0.6846
Loss: 1.498525619506836, Accuracy: 0.6846495866775513
57/57 [==============================] - 1s 12ms/step - loss: 1.5911 - accuracy: 0.6752
Loss: 1.5911248922348022, Accuracy: 0.6751946806907654
57/57 [==============================] - 1s 11ms/step - loss: 1.4971 - accuracy: 0.6674
Loss: 1.497111201286316, Accuracy: 0.6674082279205322


In [20]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6674082279205322


In [21]:
X_new = X_test[0:10]
y_preds = model.predict(X_new)
y_preds.round(2)

1/1 [==============================] - 0s 448ms/step


array([[0.  , 0.  , 1.  ],
       [0.21, 0.76, 0.03],
       [1.  , 0.  , 0.  ],
       [0.12, 0.  , 0.87],
       [0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  ],
       [0.  , 0.67, 0.33],
       [0.19, 0.  , 0.81],
       [0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  ]], dtype=float32)

In [22]:
y_test[0:10]

array([0, 1, 0, 2, 2, 0, 1, 0, 2, 0])